In [1]:
import pandas as pd
import unicodedata


In [2]:
df = pd.read_csv('../data/data.csv', encoding='utf-8')
df

,ElementId,Value
0,122430,Video: https://youtu.be/z-IwPw5gwD8 Fecha máxi...
1,69562,1. NEW. Se habilitó el Log para registrar las ...
2,106534,29.6-FIX-01: Se realizó un ajuste en el mecani...
3,52821,1. New: Se implementó el sistema de autenticac...
4,110816,30.0-FIX-01: (HRP) Se realizaron varios ajuste...
...,...,...
146,96955,R29.1-FIX-01: FIX. (EG) Se corrigió el problem...
147,97580,R29.2-FT-01: NEW. Se reescribió todo el mecan...
148,98276,R29.3-FT-01: NEW. Se creó un mecanismo para cu...
149,115966,Video: 30.2-FT-01. (HR) Ahora es posible ut...


In [3]:
df['Value'] = df['Value'].str.replace('\xa0', ' ')
df

,ElementId,Value
0,122430,Video: https://youtu.be/z-IwPw5gwD8 Fecha máxi...
1,69562,1. NEW. Se habilitó el Log para registrar las ...
2,106534,29.6-FIX-01: Se realizó un ajuste en el mecani...
3,52821,1. New: Se implementó el sistema de autenticac...
4,110816,30.0-FIX-01: (HRP) Se realizaron varios ajuste...
...,...,...
146,96955,R29.1-FIX-01: FIX. (EG) Se corrigió el problem...
147,97580,R29.2-FT-01: NEW. Se reescribió todo el mecan...
148,98276,R29.3-FT-01: NEW. Se creó un mecanismo para cu...
149,115966,Video: 30.2-FT-01. (HR) Ahora es posible ut...


In [4]:
import re
df['Value'] = df['Value'].apply(lambda x: re.sub(r'(bugs?[:\s-]?\d{0,4}:?)|((\(EG\))|(\(HRP?\))|(\(CG\))|(\(SL\))|(\(CM\)))', ' ', x, flags=re.IGNORECASE | re.MULTILINE))
df

,ElementId,Value
0,122430,Video: https://youtu.be/z-IwPw5gwD8 Fecha máxi...
1,69562,1. NEW. Se habilitó el Log para registrar las ...
2,106534,29.6-FIX-01: Se realizó un ajuste en el mecani...
3,52821,1. New: Se implementó el sistema de autenticac...
4,110816,30.0-FIX-01: Se realizaron varios ajustes de...
...,...,...
146,96955,R29.1-FIX-01: FIX. Se corrigió el problema d...
147,97580,R29.2-FT-01: NEW. Se reescribió todo el mecan...
148,98276,R29.3-FT-01: NEW. Se creó un mecanismo para cu...
149,115966,Video: 30.2-FT-01. Ahora es posible utili...


In [5]:
import re

def split_text_by_regex(row):
    # Usar una expresión regular para dividir el texto por saltos de línea o cualquier otro patrón deseado
    # \s* captura cualquier cantidad de espacios en blanco (incluidos los saltos de línea) alrededor del separador
    # Puedes ajustar la expresión regular según tus necesidades
    
    fragments = re.split(r'(?<![a-zA-Z])((\w?\d{1,2}[.:\s]\d{1,2}[-:\s]((fix)|(ft))[-\s:]?\d{0,3}[\w:.]?[\w:.]?\s{1,4})|(((fix)|(new)|(Correction))[:.\s]{0,}))(?![a-zA-Z])', row['Value'],  flags=re.IGNORECASE | re.MULTILINE)
    # Filtrar líneas vacías o con solo espacios, y quitar espacios adicionales
    #fragments = [line.strip() for line in fragments if line.strip()]
    fragments = [line.strip() for line in fragments if line is not None and line.strip() and  len(line) > 15 and not re.match(r'((\d[-.:\s])|((fix)|(ft)|(new)|(video))[-.:\s]?)', line, flags=re.IGNORECASE)]

    row['Value'] = fragments
    #print(fragments)
    return row




In [6]:
# Aplicar la función y expandir el DataFrame
df = df.apply(split_text_by_regex, axis=1)
df = df.explode('Value').reset_index(drop=True)
df

,ElementId,Value
0,122430,Ajustado error cuando se usaban suggest en el ...
1,122430,Ajustado la funcionalidad del parametro Versio...
2,122430,Ajustado error que se presentaba al enviar las...
3,122430,Se ha implementado una nueva funcionalidad que...
4,122430,Se corrigió un error en las nuevas jerarquías ...
...,...,...
2077,108410,UPDATE 2023-06-20
2078,108410,Para optimizar el envío de correos de vencimie...
2079,108410,Se realizó un ajuste en el stored procedure It...
2080,108410,Se ha realizado un ajuste en el método StoreSy...


In [7]:
from langchain.document_loaders import DataFrameLoader

loader = DataFrameLoader(df, page_content_column="Value")
data = loader.load()

In [8]:
from pprint import pprint

pprint(data[:5])

[Document(page_content='Ajustado error cuando se usaban suggest en el filtro no traía resultadosPruebas y aplicabilidad: Makro y dónde aplique, se debe probar que al usar en el filtro campos suggest, este retorne los resultados que corresponden al filtro aplicado.', metadata={'ElementId': 122430}),
 Document(page_content='Ajustado la funcionalidad del parametro VersionChangesMetaDataIdPruebas y aplicabilidad: Ebsa y dónde aplique.', metadata={'ElementId': 122430}),
 Document(page_content='Ajustado error que se presentaba al enviar las notificaciones para responder encuestas (SURVEYS II). Anteriormente, existía un límite de 12 horas para abrir la encuesta a través del enlace enviado por correo electrónico, después del límite se generaba un error. Pruebas y aplicabilidad: Seguros Mundial. Probar generando encuentas y poder usar el link para responderla despues de doce horas.', metadata={'ElementId': 122430}),
 Document(page_content="Se ha implementado una nueva funcionalidad que permite 

In [9]:
len(data)

2082

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    length_function=len,
    chunk_overlap=100
)

documents = text_splitter.split_documents(data)

In [11]:
len(documents)

2688

In [12]:
documents[0]

Document(page_content='Ajustado error cuando se usaban suggest en el filtro no traía resultadosPruebas y aplicabilidad: Makro y dónde aplique, se debe probar que al usar en el filtro campos suggest, este retorne los resultados que corresponden al filtro aplicado.', metadata={'ElementId': 122430})

In [13]:
documents[1]

Document(page_content='Ajustado la funcionalidad del parametro VersionChangesMetaDataIdPruebas y aplicabilidad: Ebsa y dónde aplique.', metadata={'ElementId': 122430})

In [14]:
from langchain.embeddings import SentenceTransformerEmbeddings

embeddings_st = SentenceTransformerEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)
## Otro modelo en español que podríamos usar es "symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli"

/home/egarces/chatbot/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
from langchain.vectorstores import Chroma
NOMBRE_INDICE_CHROMA = "../databases/embeddings-public"

In [16]:

vectorstore_chroma = Chroma.from_documents(
    documents=documents,
    embedding=embeddings_st,
    persist_directory=NOMBRE_INDICE_CHROMA
)

In [17]:
vectorstore_chroma.persist()

In [18]:
vectorstore_chroma = Chroma(
    persist_directory=NOMBRE_INDICE_CHROMA,
    embedding_function=embeddings_st
)

In [19]:
query = "Cual es el parametro para configurar el nuevo Pivot?"

docs = vectorstore_chroma.similarity_search_with_score(query, k=10)

In [20]:
docs

[(Document(page_content='(BETA): Nuevo tipo de reportes PIVOT , para configurarlos deje el parámetro Use', metadata={'ElementId': 110816}),
  8.467745780944824),
 (Document(page_content='Teniendo en cuenta la configuración en', metadata={'ElementId': 115966}),
  8.878649710363522),
 (Document(page_content='Es posible especificar por cada reporte Pivot si cuando el usuario arrastra y suelta campos, el resultado se muestre de forma contraida por defecto. Para esto configure la columna DefaultReport.AutoCollapseAll en true', metadata={'ElementId': 33641}),
  9.492097854614258),
 (Document(page_content='- Se hacen calculados del elemento padre cuando el embebido sufre una modificación. Aplicabilidad y pruebas: ver descripción del  , probar en nuevaEPS.', metadata={'ElementId': 106534}),
  9.968347549438477),
 (Document(page_content='Además, se debe configurar el parámetro ElementQueueBatchSize con el tamaño de lote de creación de elementos por cada ejecución del procesamiento de la cola.  